# Fit Ni PDF

This is an example of using the DiffPy-CMI framework to fit a Ni neutron PDF. It illustrates how to build a simple fit recipe with a PDF data file and a structure uploaded from a cif file. It should be straightforward to adapt this example to fit your own data.  Simply replace the ``dataFile`` and ``structureFile`` variables with your own and modify the ``spaceGroup`` variable as appropriate.

In [ ]:
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize.minpack import leastsq

# DiffPy-CMI modules for building a fitting recipe
from diffpy.Structure import loadStructure
from diffpy.srfit.pdf import PDFContribution
from diffpy.srfit.fitbase import FitRecipe, FitResults

from diffpy.srfit.structure import constrainAsSpaceGroup


Specify the files containing our experimental data and structure file



In [1]:
dataFile = "ni-q27r100-neutron.gr"
structureFile = "ni.cif"
spaceGroup = "Fm-3m"

 The first thing to construct is a contribution. Since this is a simple example, the contribution will simply contain our PDF data and an associated structure file. We'll give it the name "nickel".

In [ ]:
niPDF = PDFContribution("nickel")

Load the data and set the r-range over which we'll fit

In [ ]:
niPDF.loadData(dataFile)
niPDF.setCalculationRange(xmin=1, xmax=20, dx=0.01)

Add the structure from our cif file to the contribution.

In [ ]:
niStructure = loadStructure(structureFile)
niPDF.addStructure("nickel", niStructure)

The FitRecipe does the work of calculating the PDF with the fit variable that we give it.

In [ ]:
niFit = FitRecipe()

Give the PDFContribution to the FitRecipe.

In [ ]:
niFit.addContribution(niPDF)

Configure the fit variables and give them to the recipe.  We can use the srfit function constrainAsSpaceGroup to constrain the lattice and ADP parameters according to the Fm-3m space group.

In [ ]:
spaceGroupParams = constrainAsSpaceGroup(niPDF.nickel.phase, spaceGroup)
print("Space group parameters are:",
      ', '.join(p.name for p in spaceGroupParams))

We can now cycle through the parameters and activate them in the recipe as variables.

In [ ]:
for par in spaceGroupParams.latpars:
    niFit.addVar(par)

Set initial value for the ADP parameters, because CIF had no ADP data.

In [ ]:
for par in spaceGroupParams.adppars:
    niFit.addVar(par, value=0.005)

As usual, we add variables for the overall scale of the PDF and a delta2 parameter for correlated motion of neighboring atoms.

In [ ]:
niFit.addVar(niPDF.scale, 1)
niFit.addVar(niPDF.nickel.delta2, 5)

We fix Qdamp based on prior information about our beamline.


In [ ]:
niFit.addVar(niPDF.qdamp, 0.03, fixed=True)

Turn off the option to printout refinement iteration number.

In [ ]:
niFit.clearFitHooks()

We can now execute the fit using scipy's least square optimizer.

In [ ]:
print("Refine PDF using scipy's least-squares optimizer:")
print("  variables:", niFit.names)
print("  initial values:", niFit.values)

leastsq(niFit.residual, niFit.values) #running the refinement

print("  final values:", niFit.values)

Obtain and display the fit results.

In [ ]:
niResults = FitResults(niFit)
print("FIT RESULTS\n")
print(niResults)

Now, to plot the observed and refined PDF.

First, extract the data from the fit profile.

In [ ]:
r = niFit.nickel.profile.x
gobs = niFit.nickel.profile.y

Next, get the calculated PDF and compute the difference between the calculated and measured PDF.

In [ ]:
gcalc = niFit.nickel.evaluate()
baseline = 1.1 * gobs.min()
gdiff = gobs - gcalc

Finally, generate the plot.

In [ ]:
plt.figure()
plt.plot(r, gobs, 'bo', label="G(r) data",
         markerfacecolor='none', markeredgecolor='b')
plt.plot(r, gcalc, 'r-', label="G(r) fit")
plt.plot(r, gdiff + baseline, 'g-', label="G(r) diff")
plt.plot(r, np.zeros_like(r) + baseline, 'k:')
plt.xlabel(r"r ($\AA$)")
plt.ylabel(r"G ($\AA^{-2}$)")
plt.legend()
